In [2]:
# %load_ext autoreload
# %autoreload 2
import sys
sys.path.insert(0,r"/Users/feiyu/Desktop/code/NewCoEditor/src")
from coeditor.common import *
import os

import requests
import shutil
import random

os.chdir(proj_root())

当前文件路径: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py
当前目录: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor
父目录: /Users/feiyu/Desktop/code/NewCoEditor/src


In [3]:
import requests
import dateparser
from coeditor.git import GitRepo
import warnings
import time


def request_page(page: int, license: str, n_items: int = 10):
    if Path("config/github_token.txt").exists():
        token = Path("config/github_token.txt").read_text().strip()
        headers = {
            "Authorization": f"Bearer {token}"
        }
    else:
        headers = None
    return requests.get(
        f"https://api.github.com/search/repositories?q=NOT+interview+NOT+reference+NOT+course+NOT+cheatsheet+created%3A>2018-01-01+stars%3A>100+size%3A<20000+language%3ATypescript+license%3A{license}&sort=stars&order=desc&per_page={n_items}&page={page}",
        headers=headers,
    ).json()


def fetch_python_repos(license2counts: dict[str, int]):
    n_repos = sum(license2counts.values())
    repos = dict[str, GitRepo]()
    with tqdm(total=n_repos) as pbar:
        for license, n_repos in license2counts.items():
            for i in range(1, n_repos // 100 + 1):
                page = request_page(i, n_items=100, license=license)
                if (msg := page.get("message", "")) and msg.startswith(
                    "API rate limit exceeded"
                ):
                    print("API rate limit exceeded, now wait for 1 min")
                    time.sleep(60)
                    continue
                if not page.get("items"):
                    print("Fetching page failed:")
                    print(page)
                    break
                for item in page["items"]:
                    r = GitRepo.from_github_item(item)
                    if not r.archived:
                        if r.authorname() in repos:
                            print(f"[warning] {r.authorname()} already in repos")
                        repos[r.authorname()] = r
                pbar.update(len(page["items"]))
    return [repos[k] for k in list(repos)]


In [4]:
{
    l: int(request_page(0, l, n_items=1)["total_count"])
    for l in ["mit", "apache-2.0", "bsd-3-clause", "bsd-2-clause"]
}


{'mit': 9636, 'apache-2.0': 1173, 'bsd-3-clause': 127, 'bsd-2-clause': 19}

In [5]:
license2counts = {
    "mit": 1000,
    "apache-2.0": 1000,
    "bsd-3-clause": 500,
}

all_repos = fetch_python_repos(license2counts)
print("Repos:", len(all_repos))

 20%|██        | 500/2500 [00:13<00:56, 35.41it/s]

API rate limit exceeded, now wait for 1 min


 60%|██████    | 1500/2500 [01:41<00:35, 28.39it/s]

API rate limit exceeded, now wait for 1 min


 77%|███████▋  | 1927/2500 [02:55<00:52, 10.96it/s]

Fetching page failed:
{'total_count': 127, 'incomplete_results': False, 'items': []}
Repos: 1798


In [6]:
# !筛选repo

all_repos=all_repos[:3]

In [7]:
dataset_name = "perm2k_ts"  # permissive licensed 2K repos
repos_dir = get_dataset_dir(dataset_name)
(repos_dir / "downloading").mkdir(exist_ok=True, parents=True)
(repos_dir / "downloaded").mkdir(exist_ok=True, parents=True)

downloaded = pmap(
    GitRepo.download,
    all_repos,
    key_args={"repos_dir": repos_dir, "full_history": True},
    desc="downloading repos",
    max_workers=4,
    chunksize=1,
)

print("Successfully downloaded:", sum(downloaded))
downloaded_repos = [r for r, d in zip(all_repos, downloaded) if d]


downloading repos:   0%|          | 0/3 [00:00<?, ?it/s]

当前文件路径: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py当前文件路径:
 当前目录:/Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py 
/Users/feiyu/Desktop/code/NewCoEditor/src/coeditor
当前目录: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor
父目录:当前文件路径:父目录: /Users/feiyu/Desktop/code/NewCoEditor/src
  /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py/Users/feiyu/Desktop/code/NewCoEditor/src

当前目录: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor
父目录: /Users/feiyu/Desktop/code/NewCoEditor/src


downloading repos: 100%|██████████| 3/3 [00:05<00:00,  1.68s/it]

Successfully downloaded: 3


In [8]:
print("Successfully downloaded:", len(downloaded_repos))

Successfully downloaded: 3


In [9]:
# now filter out repos with less than 50 commits
filtered_repos = [r for r in tqdm(downloaded_repos) if r.count_commits(repos_dir) >= 50]
print("After filtering by commits:", len(filtered_repos))

100%|██████████| 3/3 [00:00<00:00, 23.33it/s]

After filtering by commits: 3


In [10]:
from coeditor.dataset import get_repo_signature

repo_paths = [repos_dir / "downloaded" / r.authorname() for r in filtered_repos]
sigs = pmap(get_repo_signature, repo_paths, desc="getting repo signatures", chunksize=1)
sig_groups = groupby(enumerate(sigs), lambda x: x[1])

duplicates = set[str]()
for sig, group in sig_groups.items():
    if len(group) > 1:
        print(f"{len(group)} repos have the same signature {sig}:")
        for i, _ in group:
            print(f"  {downloaded_repos[i].authorname()}")
        for i, _ in group[1:]:
            duplicates.add(downloaded_repos[i].authorname())

print("Totoal duplicates:", len(duplicates))
filtered_repos = [r for r in filtered_repos if r.authorname() not in duplicates]
print("After filtering duplicates:", len(filtered_repos))


/opt/miniconda3/envs/guohx_coeditor/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
getting repo signatures:   0%|          | 0/3 [00:00<?, ?it/s]

当前文件路径: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py
当前文件路径:当前目录:  /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py/Users/feiyu/Desktop/code/NewCoEditor/src/coeditor

父目录: /Users/feiyu/Desktop/code/NewCoEditor/src
当前文件路径:当前目录: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor/common.py
当前目录: /Users/feiyu/Desktop/code/NewCoEditor/src/coeditor 
/Users/feiyu/Desktop/code/NewCoEditor/src/coeditor父目录:
 /Users/feiyu/Desktop/code/NewCoEditor/src
父目录: /Users/feiyu/Desktop/code/NewCoEditor/src


getting repo signatures: 100%|██████████| 3/3 [01:32<00:00, 30.72s/it]

Totoal duplicates: 0
After filtering duplicates: 3


In [11]:
n_test = 1 # !
n_valid = 1
n_train = len(filtered_repos) - n_test - n_valid
print(f"n_test={n_test}, n_valid={n_valid}, n_train={n_train}")

random.seed(42)
filtered_repos.sort(key=lambda r: r.authorname())
random.shuffle(filtered_repos)

split = {
    "test": filtered_repos[:n_test],
    "valid": filtered_repos[n_test : n_test + n_valid],
    "train": filtered_repos[n_test + n_valid :][:n_train],
}

pickle_dump(repos_dir / "repos_split.pkl", split)


n_test=1, n_valid=1, n_train=1


In [12]:
# move downloaded repos to their split group
for group, rs in split.items():
    for repo in tqdm(rs, desc=f"moving {group}"):
        dest = repos_dir / "repos" / group
        dest.mkdir(exist_ok=True, parents=True)
        shutil.move(repos_dir / "downloaded" / repo.authorname(), dest)


moving train: 100%|██████████| 1/1 [00:00<00:00, 680.67it/s]
